Bagging with decision trees: training

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


rng = np.random.RandomState(seed=4190)

def bagging_fit(X, y, n_estimators, max_depth=5, max_samples=200): # Initialize a random seed
    n_examples = len(y)
    estimators = [DecisionTreeClassifier(max_depth=max_depth) # Creates a list of untrained base estimators
    for _ in range(n_estimators)]
    for tree in estimators:
        bag = np.random.choice(n_examples, max_samples, # Generate a boostrap sample
        replace=True)
        tree.fit(X[bag, :], y[bag]) # Fits a tree to the boostrap sample
    return estimators

Bagging with decision trees: prediction

In [ ]:
from scipy.stats import mode


def bagging_predict(X, estimators):
    # Predicts each test example using each estimator in the ensemble
    all_predictions = np.array([tree.predict(X) for tree in estimators])
    # Makes the final predictions by majority voting
    ypred, _ = mode(all_predictions, axis=0, keepdims=False)
    return np.squeeze(ypred)

Implementation EXAMPLE 

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X, y = make_moons(n_samples=300, noise=.25, random_state=rng)

Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.33, random_state=rng)

# Trains a bagging ensemble
bag_ens = bagging_fit(Xtrn, ytrn, n_estimators=500, max_depth=12, max_samples=300)

# Makes the final predictions by majority voting
ypred = bagging_predict(Xtst, bag_ens)

print(accuracy_score(ytst, ypred))

## Bagging (with Scikit-learn)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


# Sets the base-learning algorithm along with hyperparameters
base_estimator = DecisionTreeClassifier(max_depth=10)

bag_ens = BaggingClassifier(base_estimator=base_estimator,
                            n_estimators=500,   # Trains 500 base estimators
                            max_samples=100,    # Each base estimator will be trained on a bootstrap sample of size 100.
                            oob_score=True,     # Uses an OOB sample to estimate the generalization
                            random_state=rng)   # Tip: n_jobs=–1 usa todos los cpus del pc

bag_ens.fit(Xtrn, ytrn)
ypred = bag_ens.predict(Xtst)

TIP: El embolsado es más efectivo con clasificadores complejos y no lineales que
tienden a sobreajustar los datos. Estos modelos complejos y sobreajustados son inestables, por lo que
es decir, altamente sensible a pequeñas variaciones en los datos de entrenamiento. Para ver por qué, considere que los árboles de decisión individuales en un conjunto en bolsa tienen aproximadamente la misma complejidad. Sin embargo, debido al muestreo de arranque, se han entrenado en diferentes réplicas del conjunto de datos y sobreajustado de manera diferente. De otra manera, todos se sobreajustan aproximadamente en la misma cantidad, pero en diferentes lugares. El embolsado funciona mejor con tales modelos porque su agregación de modelos reduce el sobreajuste, lo que en última instancia conduce a un conjunto más robusto y estable.

In [ ]:
bag_ens.oob_score_

In [ ]:
accuracy_score(ytst, ypred)

### Random Forests (with scikit-learn)

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf_ens = RandomForestClassifier(n_estimators=500,
                                max_depth=10,
                                oob_score=True,
                                n_jobs=-1,
                                random_state=rng)

rf_ens.fit(Xtrn, ytrn)

ypred = rf_ens.predict(Xtst) # Uses an OOB sample to estimate the generalization error

Feature Seleccion:

La selección de características, también conocida como selección de subconjuntos variables, es un procedimiento para identificar clasificando las características/atributos de datos más influyentes o relevantes. La selección de características es un paso importante del proceso de modelado, especialmente para datos de alta dimensión.
Descartar las características menos relevantes a menudo mejora el rendimiento de la generalización y minimiza el sobreajuste. También suele mejorar la eficiencia computacional del entrenamiento.
Estas preocupaciones son consecuencias de la maldición de la dimensionalidad, donde un gran número de características puede inhibir la capacidad de un modelo para generalizar de manera efectiva.

In [ ]:
# Obtain feature importances for the simple 2D data set
for i, score in enumerate(rf_ens.feature_importances_):
    print('Feature x{0}: {1:6.5f}'.format(i, score))

### Pasting

Pasting: El embolsado (Bagging) utiliza muestreo de arranque (Boostrap) o muestreo con reemplazo. Si, en cambio, muestreamos subconjuntos para entrenamiento sin reemplazo, tenemos una variante de embolsado conocida como pegado (Pasting). El pegado se diseñó para conjuntos de datos muy grandes, donde no es necesario el muestreo con reemplazo

Tip: BaggingClassifier se puede ampliar fácilmente para realizar el PASTING configurando bootstrap=False y convirtiéndolo en pequeños subconjuntos de submuestra para el entrenamiento configurando max_samples en una pequeña fracción, digamos max_samples=0.05.

### Random Subspace and Random Patches

<img src="img/random_xx.jpg">

Nota: Random Subsapece y Random Patche NO usan necesariamente los arboles de decision, pueden usar cualquier algoritmo de ML

## Extra Trees

El algoritmo Extra Trees (Extreme Randomized Trees) es una variante del algoritmo de Random Forest en aprendizaje automático. Es un algoritmo de aprendizaje supervisado utilizado tanto para clasificación como para regresión.

La principal característica distintiva de Extra Trees es que se aplica una estrategia de selección aleatoria más agresiva para construir los árboles de decisión en el bosque. Mientras que en Random Forest se seleccionan las mejores divisiones entre un subconjunto de características para cada nodo del árbol, en Extra Trees se eligen divisiones aleatorias sin tener en cuenta la importancia relativa de las características. Esto resulta en una mayor aleatoriedad y diversidad en los árboles construidos.